# Defining a grid from scratch

In this example we are going to create a grid just by using GrdiCal's comands and we will run a power flow study.

In [116]:
import pandas as pd
import numpy as np

from GridCal.Engine.Devices.branch import Branch, BranchTemplate
from GridCal.Engine.Devices.generator import Generator
from GridCal.Engine.Devices.load import Load
from GridCal.Engine.Devices.types import BranchType
from GridCal.Engine.Simulations.PowerFlow.steady_state.power_flow_driver import \
    PowerFlowOptions, SolverType, PowerFlow
from GridCal.Engine.Devices.bus import Bus
from GridCal.Engine.Core.multi_circuit import MultiCircuit
from tabulate import tabulate

%matplotlib inline

Let's create a new grid object:

In [117]:
grid = MultiCircuit(name='lynn 5 bus')

Define this first bus with all the parameters for demonstration purposes:

In [118]:
bus1 = Bus(
    name='Bus1',
    vnom=10,   # Nominal voltage in kV
    vmin=0.9,  # Bus minimum voltage in per unit
    vmax=1.1,  # Bus maximum voltage in per unit
    xpos=0,    # Bus x position in pixels
    ypos=0,    # Bus y position in pixels
    height=0,  # Bus height in pixels
    width=0,   # Bus width in pixels
    active=True,   # Is the bus active?
    is_slack=False,  # Is this bus a slack bus?
    area='Default',  # Area (for grouping purposes only)
    zone='Default',  # Zone (for grouping purposes only)
    substation='Default'  # Substation (for grouping purposes only)
)

The rest of the buses are defined with the default parameters:

In [119]:
bus2 = Bus(name='Bus2')
bus3 = Bus(name='Bus3')
bus4 = Bus(name='Bus4')
bus5 = Bus(name='Bus5')

Add the bus objects to the circuit:

In [120]:
grid.add_bus(bus1)
grid.add_bus(bus2)
grid.add_bus(bus3)
grid.add_bus(bus4)
grid.add_bus(bus5)

In GridCal, the loads, generators etc. are stored within each bus object.

We'll define the first load completely:

In [121]:
l2 = Load(
    name='Load',
    G=0,  # Impedance of the ZIP model in MVA at the nominal voltage
    B=0,
    Ir=0,
    Ii=0,  # Current of the ZIP model in MVA at the nominal voltage
    P=40,
    Q=20,  # Power of the ZIP model in MVA
    P_prof=None,  # Impedance profile
    Q_prof=None,  # Current profile
    Ir_prof=None,  # Power profile
    Ii_prof=None,
    G_prof=None,
    B_prof=None,
    active=True,  # Is active?
    mttf=0.0,  # Mean time to failure
    mttr=0.0  # Mean time to recovery
)
grid.add_load(bus2, l2)

Define the others with the default parameters:

In [122]:
grid.add_load(bus3, Load(P=25, Q=15))
grid.add_load(bus4, Load(P=40, Q=20))
grid.add_load(bus5, Load(P=50, Q=20))

In [123]:
g1 = Generator(
    name='gen',
    active_power=0.0,  # Active power in MW, since this generator is used to set the slack , is 0
    voltage_module=1.0,  # Voltage set point to control
    Qmin=-9999,  # minimum reactive power in MVAr
    Qmax=9999,  # Maximum reactive power in MVAr
    Snom=9999,  # Nominal power in MVA
    power_prof=None,  # power profile
    vset_prof=None,  # voltage set point profile
    active=True  # Is active?
)
grid.add_generator(bus1, g1)

In [124]:
br1 = Branch(
    bus_from=bus1,
    bus_to=bus2,
    name='Line 1-2',
    r=0.05,  # resistance of the pi model in per unit
    x=0.11,  # reactance of the pi model in per unit
    g=1e-20,  # conductance of the pi model in per unit
    b=0.02,  # susceptance of the pi model in per unit
    rate=50,  # Rate in MVA
    tap=1.0,  # Tap value (value close to 1)
    shift_angle=0,  # Tap angle in radians
    active=True,  # is the branch active?
    mttf=0,  # Mean time to failure
    mttr=0,  # Mean time to recovery
    branch_type=BranchType.Line,  # Branch type tag
    length=1,  # Length in km (to be used with templates)
    template=BranchTemplate()  # Branch template (The default one is void)
)
grid.add_branch(br1)

All other branches with necessary parameters only:

In [125]:
grid.add_branch(Branch(bus1, bus3, name='Line 1-3', r=0.05, x=0.11, b=0.02, rate=50))
grid.add_branch(Branch(bus1, bus5, name='Line 1-5', r=0.03, x=0.08, b=0.02, rate=80))
grid.add_branch(Branch(bus2, bus3, name='Line 2-3', r=0.04, x=0.09, b=0.02, rate=3))
grid.add_branch(Branch(bus2, bus5, name='Line 2-5', r=0.04, x=0.09, b=0.02, rate=10))
grid.add_branch(Branch(bus3, bus4, name='Line 3-4', r=0.06, x=0.13, b=0.03, rate=30))
grid.add_branch(Branch(bus4, bus5, name='Line 4-5', r=0.04, x=0.09, b=0.02, rate=30))

Define power flow options:

In [126]:
power_flow_options = PowerFlowOptions(
    solver_type=SolverType.NR,  # Base method to use
    verbose=False,  # Verbose option where available
    tolerance=1e-6,  # power error in p.u.
    max_iter=25,  # maximum iteration number
    control_q=True  # if to control the reactive power
)

Declare and execute the power flow simulation

In [127]:
power_flow = PowerFlow(grid, power_flow_options)
power_flow.run()

Now, let's compose a data frame with the voltage results:

In [128]:
headers = ['Vm (p.u.)', 'Va (Deg)', 'Vre', 'Vim']
Vm = np.abs(power_flow.results.voltage)
Va = np.angle(power_flow.results.voltage, deg=True)
Vre = power_flow.results.voltage.real
Vim = power_flow.results.voltage.imag
data = np.c_[Vm, Va, Vre, Vim]
v_df = pd.DataFrame(data=data, columns=headers, index=grid.bus_names)
print('\n', v_df)


       Vm (p.u.)  Va (Deg)       Vre       Vim
Bus1   1.000000  0.000000  1.000000  0.000000
Bus2   0.955306 -2.404835  0.954465 -0.040085
Bus3   0.954818 -2.363804  0.954005 -0.039381
Bus4   0.933334 -3.649786  0.931441 -0.059414
Bus5   0.953394 -2.688959  0.952344 -0.044728


Let's do the same for the branch results:

In [129]:
headers = ['Loading (%)', 'Current(p.u.)', 'Power (MVA)']
loading = np.abs(power_flow.results.loading) * 100
current = np.abs(power_flow.results.Ibranch)
power = np.abs(power_flow.results.Sbranch)
data = np.c_[loading, current, power]
br_df = pd.DataFrame(data=data, columns=headers, index=grid.branch_names)
print('\n', br_df)


           Loading (%)  Current(p.u.)  Power (MVA)
Line 1-2    99.610256       0.498051    49.805128
Line 1-3    99.395638       0.496978    49.697819
Line 1-5    95.072476       0.760580    76.057981
Line 2-3    55.519737       0.017444     1.665592
Line 2-5    50.626874       0.052995     5.062687
Line 3-4    65.559073       0.205984    19.667722
Line 4-5    81.043272       0.255015    24.312982


Finally, print the execution metrics:

In [130]:
print('\nError:', power_flow.results.error)
print('Elapsed time (s):', power_flow.results.elapsed, '\n')

print(tabulate(v_df, tablefmt="pipe", headers=v_df.columns.values))
print()
print(tabulate(br_df, tablefmt="pipe", headers=br_df.columns.values))


Error: [1.8965826464878432e-08]
Elapsed time (s): [0.016323328018188477] 

|      |   Vm (p.u.) |   Va (Deg) |      Vre |        Vim |
|:-----|------------:|-----------:|---------:|-----------:|
| Bus1 |    1        |    0       | 1        |  0         |
| Bus2 |    0.955306 |   -2.40484 | 0.954465 | -0.0400846 |
| Bus3 |    0.954818 |   -2.3638  | 0.954005 | -0.0393809 |
| Bus4 |    0.933334 |   -3.64979 | 0.931441 | -0.0594139 |
| Bus5 |    0.953394 |   -2.68896 | 0.952344 | -0.0447275 |

|          |   Loading (%) |   Current(p.u.) |   Power (MVA) |
|:---------|--------------:|----------------:|--------------:|
| Line 1-2 |       99.6103 |       0.498051  |      49.8051  |
| Line 1-3 |       99.3956 |       0.496978  |      49.6978  |
| Line 1-5 |       95.0725 |       0.76058   |      76.058   |
| Line 2-3 |       55.5197 |       0.0174441 |       1.66559 |
| Line 2-5 |       50.6269 |       0.0529954 |       5.06269 |
| Line 3-4 |       65.5591 |       0.205984  |      19.6677  |